# PCA

### In this notebook we tackle the problem of sparsity in the feature representations of the different modes (chromatin accessibility, gene expression, surface protein levels). 

As noted in https://www.kaggle.com/code/leohash/complete-eda-of-mmscel-integration-data/notebook, DNA data has between 1-30k of the 229k features being nonzero, RNA data has 2-8k of the ~28k features as nonzero, and protein data has a small number of features and is sparse, which means this notebook don't care :)

We will apply PCA (and maybe some other techniques) to investigate whether we can usefully hop into a lower-dimensional, densely-populated representation for either of these two modes.

In [1]:
# imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datasets import SparseDataset, H5Dataset

from sklearn.decomposition import PCA, SparsePCA

In [2]:
# -----------------------------------------------------------

# Da Work

## DNA (Chromatin Accessibility)

In [3]:
train_f = FP_MULTIOME_TRAIN_INPUTS
test_f = FP_MULTIOME_TEST_INPUTS

max_n_components = 20000

p_dna = PCA(max_n_components)
p_dna.fit(train_f)
p_dna.fit(test_f)

NameError: name 'FP_MULTIOME_TRAIN_INPUTS' is not defined

## RNA (Gene Expression)

In [4]:
import pandas as pd

multi_df = pd.read_hdf('data/train_multi_targets.h5', start=1000, stop=2000)
cite_df = pd.read_hdf('data/train_cite_inputs.h5', start=1000, stop=2000)

multi_keys = list(multi_df.keys())
cite_keys = list(cite_df.keys())

for i in range(len(cite_keys)):
    cite_keys[i] = cite_keys[i].split('_')[0]

multi_idxs = []
cite_idxs = []
for i, s in enumerate(multi_keys):
    if s in cite_keys:
        multi_idxs.append(i)
        cite_idxs.append(cite_keys.index(s))

In [ ]:
multi_rna = np.asarray(H5Dataset('all', 'multi').targets_h5)
cite_rna = np.asarray(H5Dataset('all', 'cite').inputs_h5)
multi_shared = multi_rna[:, multi_idxs]
cite_shared = cite_rna[:, cite_idxs]
shared = np.concatenate((multi_shared, cite_shared), axis=0)

In [ ]:
p = PCA(100)
p.fit(shared[:105])